# Tutorial 05: Reflection

In this tutorial, you'll learn how to build self-critiquing agents that iteratively improve their outputs through reflection.

**What you'll learn:**
- **Reflection loops**: Generate → Critique → Revise
- **Self-improvement**: Using LLM to evaluate its own outputs
- **Iteration control**: When to stop refining
- **Quality enhancement**: Producing better outputs through feedback

By the end, you'll have an agent that writes, critiques, and revises its own work.

## What is Reflection?

Reflection is a pattern where an LLM:
1. **Generates** an initial output
2. **Critiques** its own work (or has another LLM critique it)
3. **Revises** based on the critique
4. **Repeats** until satisfied

This mirrors how humans improve their work through drafts and revisions.

### Use Cases

- **Writing**: Essays, reports, documentation
- **Code**: Generate, review, refactor
- **Analysis**: Initial assessment, deeper analysis, conclusions
- **Problem-solving**: Solution, evaluation, refinement

## Graph Visualization

![Reflection Graph](../docs/images/05-reflection-graph.png)

The graph cycles between generation and critique until the output is satisfactory.

In [ ]:
# Setup
from langgraph_ollama_local import LocalAgentConfig

config = LocalAgentConfig()
print(f"Ollama: {config.ollama.base_url}")
print(f"Model: {config.ollama.model}")

## Step 1: Define State

Our reflection state tracks:
- Messages (conversation history)
- Current draft
- Latest critique
- Iteration count

In [ ]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph.message import add_messages

class ReflectionState(TypedDict):
    """State for reflection loop."""
    messages: Annotated[list, add_messages]  # Conversation history
    task: str                                 # Original task
    draft: str                                # Current draft
    critique: str                             # Latest critique
    iteration: int                            # Current iteration

print("State defined with: messages, task, draft, critique, iteration")

## Step 2: Create the LLM

In [ ]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model=config.ollama.model,
    base_url=config.ollama.base_url,
    temperature=0.7,  # Some creativity for writing
)

print("LLM configured")

## Step 3: Define the Generator Node

The generator creates or revises the draft:

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

GENERATOR_PROMPT = """You are a skilled writer. Your task is to write or revise content based on feedback.

If this is the first draft, write a complete response to the task.
If you have received critique, revise your draft to address the feedback.

Focus on:
- Clarity and conciseness
- Accuracy and completeness
- Engaging and professional tone
"""

def generate_node(state: ReflectionState) -> dict:
    """Generate or revise the draft."""
    iteration = state.get("iteration", 0)
    task = state["task"]
    draft = state.get("draft", "")
    critique = state.get("critique", "")
    
    if iteration == 0:
        # First draft
        user_msg = f"Write a response to this task:\n\n{task}"
    else:
        # Revision based on critique
        user_msg = f"""Revise this draft based on the critique.

ORIGINAL TASK: {task}

CURRENT DRAFT:
{draft}

CRITIQUE:
{critique}

Please provide an improved version addressing the feedback."""
    
    messages = [
        SystemMessage(content=GENERATOR_PROMPT),
        HumanMessage(content=user_msg)
    ]
    
    response = llm.invoke(messages)
    new_draft = response.content
    
    print(f"\n=== Generation (iteration {iteration + 1}) ===")
    print(new_draft[:200] + "..." if len(new_draft) > 200 else new_draft)
    
    return {
        "draft": new_draft,
        "iteration": iteration + 1,
        "messages": [AIMessage(content=f"Draft {iteration + 1}: {new_draft}")]
    }

## Step 4: Define the Critique Node

The critique node evaluates the draft and provides feedback:

In [ ]:
CRITIQUE_PROMPT = """You are a thoughtful editor. Review the draft and provide constructive feedback.

Evaluate:
1. Does it fully address the task?
2. Is it clear and well-organized?
3. Are there any errors or areas for improvement?
4. Is the tone appropriate?

If the draft is excellent and needs no changes, respond with exactly: "APPROVED"
Otherwise, provide specific, actionable feedback for improvement.
"""

def critique_node(state: ReflectionState) -> dict:
    """Critique the current draft."""
    task = state["task"]
    draft = state["draft"]
    iteration = state["iteration"]
    
    user_msg = f"""Review this draft for the given task.

TASK: {task}

DRAFT:
{draft}

Provide your critique or respond with "APPROVED" if no changes are needed."""
    
    messages = [
        SystemMessage(content=CRITIQUE_PROMPT),
        HumanMessage(content=user_msg)
    ]
    
    response = llm.invoke(messages)
    critique = response.content
    
    print(f"\n=== Critique (iteration {iteration}) ===")
    print(critique[:200] + "..." if len(critique) > 200 else critique)
    
    return {
        "critique": critique,
        "messages": [AIMessage(content=f"Critique {iteration}: {critique}")]
    }

## Step 5: Define the Routing Logic

Decide whether to continue revising or finish:

In [ ]:
MAX_ITERATIONS = 3

def should_continue(state: ReflectionState) -> str:
    """Decide whether to continue refining or finish."""
    iteration = state["iteration"]
    critique = state.get("critique", "")
    
    # Stop if approved
    if "APPROVED" in critique.upper():
        print("\n✓ Draft approved!")
        return "end"
    
    # Stop if max iterations reached
    if iteration >= MAX_ITERATIONS:
        print(f"\n✓ Max iterations ({MAX_ITERATIONS}) reached")
        return "end"
    
    print(f"\n→ Continuing to iteration {iteration + 1}")
    return "generate"

print(f"Max iterations: {MAX_ITERATIONS}")

## Step 6: Build the Graph

In [ ]:
from langgraph.graph import StateGraph, START, END

# Build the reflection graph
workflow = StateGraph(ReflectionState)

# Add nodes
workflow.add_node("generate", generate_node)
workflow.add_node("critique", critique_node)

# Add edges
workflow.add_edge(START, "generate")       # Start by generating
workflow.add_edge("generate", "critique")  # Then critique
workflow.add_conditional_edges(
    "critique",
    should_continue,
    {"generate": "generate", "end": END}
)

# Compile
graph = workflow.compile()

print("Reflection graph compiled!")

In [ ]:
# Visualize
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception as e:
    print(f"Could not render: {e}")

## Step 7: Run the Reflection Loop

In [ ]:
# Run the reflection loop
task = "Write a brief explanation of what LangGraph is and why it's useful for building AI agents."

print(f"Task: {task}")
print("="*60)

result = graph.invoke({
    "task": task,
    "messages": [],
    "draft": "",
    "critique": "",
    "iteration": 0
})

print("\n" + "="*60)
print("FINAL DRAFT:")
print("="*60)
print(result["draft"])

## Step 8: Test with Different Tasks

In [ ]:
def reflect_on_task(task: str) -> str:
    """Run reflection loop on a task and return final draft."""
    result = graph.invoke({
        "task": task,
        "messages": [],
        "draft": "",
        "critique": "",
        "iteration": 0
    })
    return result["draft"]

# Test with a different task
email_task = "Write a professional email declining a meeting invitation due to a scheduling conflict."

print(f"Task: {email_task}")
print("="*60)

final_email = reflect_on_task(email_task)

print("\n" + "="*60)
print("FINAL EMAIL:")
print("="*60)
print(final_email)

## Complete Code

In [ ]:
# Complete Reflection Agent

from typing import Annotated
from typing_extensions import TypedDict
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_ollama import ChatOllama
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph_ollama_local import LocalAgentConfig

# === State ===
class ReflectionState(TypedDict):
    messages: Annotated[list, add_messages]
    task: str
    draft: str
    critique: str
    iteration: int

# === LLM ===
config = LocalAgentConfig()
llm = ChatOllama(
    model=config.ollama.model,
    base_url=config.ollama.base_url,
    temperature=0.7,
)

# === Nodes ===
def generate(state: ReflectionState) -> dict:
    iteration = state.get("iteration", 0)
    if iteration == 0:
        prompt = f"Write a response: {state['task']}"
    else:
        prompt = f"Revise based on critique:\nDraft: {state['draft']}\nCritique: {state['critique']}"
    
    response = llm.invoke([HumanMessage(content=prompt)])
    return {"draft": response.content, "iteration": iteration + 1}

def critique(state: ReflectionState) -> dict:
    prompt = f"Critique this draft (say APPROVED if perfect):\n{state['draft']}"
    response = llm.invoke([HumanMessage(content=prompt)])
    return {"critique": response.content}

def should_continue(state: ReflectionState) -> str:
    if "APPROVED" in state.get("critique", "").upper():
        return "end"
    if state["iteration"] >= 3:
        return "end"
    return "generate"

# === Graph ===
workflow = StateGraph(ReflectionState)
workflow.add_node("generate", generate)
workflow.add_node("critique", critique)
workflow.add_edge(START, "generate")
workflow.add_edge("generate", "critique")
workflow.add_conditional_edges("critique", should_continue, {"generate": "generate", "end": END})
graph = workflow.compile()

# === Use ===
result = graph.invoke({"task": "Explain recursion in 2 sentences.", "messages": [], "draft": "", "critique": "", "iteration": 0})
print(result["draft"])

## Key Concepts Recap

| Concept | Description |
|---------|-------------|
| **Generate** | Create initial or revised content |
| **Critique** | Evaluate and provide feedback |
| **Conditional Edge** | Route based on critique result |
| **Iteration Limit** | Prevent infinite loops |
| **Approval Signal** | Explicit signal to stop refining |

## Variations

1. **Two-model reflection**: Use a stronger model for critique
2. **Structured feedback**: Use JSON for specific improvement areas
3. **Tool-augmented**: Add research tools to fact-check

## What's Next?

In [Tutorial 06: Plan and Execute](06_plan_and_execute.ipynb), you'll learn:
- Breaking complex tasks into steps
- Planning before executing
- Re-planning based on results